In [1]:
import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [13]:
train

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival
...,...,...
9998,You provide support in what countries?,country_support
9999,What countries are you supporting?,country_support
10000,What countries are getting support?,country_support
10001,Are cards available in the EU?,country_support


In [24]:
# Separate train labels and text
train_labels = train['category']
train_text = train['text']

# Separate test labels and text
test_labels = test['category']
test_text = test['text']

In [8]:
model_id = 'philschmid/BERT-Banking77'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

# set device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# put model on GPU (is done in-place)
model.to(device)

# List to store predicted probabilities
predicted_probabilities = []

# Iterate through each sentence
for sentence in test['text']:
    # Tokenize sentence
    encoding = tokenizer(sentence, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    # Move input tensors to GPU if available
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Retrieve predicted probabilities
    probabilities = torch.softmax(outputs.logits, dim=1).cpu().tolist()[0]
    predicted_probabilities.append(probabilities)
    
    # Compare predicted labels with ground truth labels
    predicted_labels = [label for label, _ in label_prob_pairs]
    true_labels = true_labels.split(',')
    
    if predicted_labels != true_labels:
        incorrect_predictions.append((sentence, true_labels, predicted_labels))

# Output the predicted probability scores for all label possibilities
for i, sentence in enumerate(test['text']):
    print("Sentence:", sentence)
    print("Predicted Probabilities:")
    print(predicted_probabilities[i])
    print()

# # forward pass
# outputs = model(**encoding)

# # get predicted class indices
# predicted_class_indices = outputs.logits.argmax(-1).tolist()
# # turn into actual class names
# predicted_classes = [model.config.id2label[label] for label in predicted_class_indices]
# print(predicted_classes)

KeyboardInterrupt: 

In [22]:
train_text

,text
0,I am still waiting on my card?
1,What can I do if my card still hasn't arrived ...
2,I have been waiting over a week. Is the card s...
3,Can I track my card while it is in the process...
4,"How do I know if I will get my card, or if it ..."
...,...
9998,You provide support in what countries?
9999,What countries are you supporting?
10000,What countries are getting support?
10001,Are cards available in the EU?
